In [1]:
import pandas as pd
import numpy as np
import time

/home/coea/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# # data = pd.read_csv("D:/Udler downloads/train.csv")
data = pd.read_csv("../../../datasets/Churn_Modelling.csv")

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [5]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [8]:
def load_and_standardize_data(path):
    # read in from csv
    df = pd.read_csv(path, sep=',')
    # replace nan with -99
    # df = df.fillna(-99)
    y = df['Exited']
    df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
       'Gender','Exited'],axis=1,inplace=True)
    xtrain,xtest,ytrain,ytest = train_test_split(df,y,stratify = y,test_size = 0.3,random_state = 4)
    # df = df.values.reshape(-1, df.shape[1]).astype('float32')
    # randomly split

    # standardize values
#     scaler = preprocessing.StandardScaler()
    scaler=preprocessing.MinMaxScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest= scaler.transform(xtest)
    return xtrain,xtest,scaler

In [9]:
from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path, train=True):
        self.X_train, self.X_test, self.standardizer = load_and_standardize_data(DATA_PATH)
        if train:
            # self.x = torch.from_numpy(self.X_train)
            self.x = torch.FloatTensor(self.X_train)
            self.len=self.x.shape[0]
        else:
            # self.x = torch.from_numpy(self.X_test)
            self.x = torch.FloatTensor(self.X_test)
            self.len=self.x.shape[0]
        del self.X_train
        del self.X_test
    def __getitem__(self,index):
        return self.x[index]
    def __len__(self):
        return self.len

In [10]:
traindata_set=DataBuilder(DATA_PATH, train=True)
testdata_set=DataBuilder(DATA_PATH, train=False)

trainloader=DataLoader(dataset=traindata_set,batch_size=20000)
testloader=DataLoader(dataset=testdata_set,batch_size=20000)

In [11]:
type(trainloader.dataset.x), type(testloader.dataset.x)

(torch.Tensor, torch.Tensor)

In [12]:
trainloader.dataset.x.shape, testloader.dataset.x.shape

(torch.Size([7000, 8]), torch.Size([3000, 8]))

In [13]:
trainloader.dataset.x

tensor([[0.8700, 0.2162, 0.9000,  ..., 1.0000, 1.0000, 0.3430],
        [0.8600, 0.2568, 1.0000,  ..., 1.0000, 0.0000, 0.6317],
        [0.6340, 0.2432, 0.3000,  ..., 1.0000, 1.0000, 0.9344],
        ...,
        [0.3860, 0.2568, 0.3000,  ..., 1.0000, 1.0000, 0.3946],
        [0.4180, 0.3243, 0.7000,  ..., 1.0000, 1.0000, 0.9504],
        [0.7880, 0.3378, 1.0000,  ..., 0.0000, 1.0000, 0.1211]])

In [14]:
testloader.dataset.x

tensor([[0.7860, 0.2973, 0.8000,  ..., 1.0000, 0.0000, 0.4745],
        [0.2760, 0.0541, 0.6000,  ..., 1.0000, 1.0000, 0.3320],
        [0.5320, 0.3378, 0.3000,  ..., 1.0000, 0.0000, 0.0938],
        ...,
        [0.5060, 0.1757, 0.8000,  ..., 1.0000, 1.0000, 0.8498],
        [0.5540, 0.3514, 0.6000,  ..., 0.0000, 0.0000, 0.1765],
        [0.5280, 0.2432, 0.1000,  ..., 1.0000, 0.0000, 0.7308]])

In [15]:
latent_dims = 2
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

## WNN Functions

In [16]:
1/np.sqrt(3)

0.5773502691896258

In [17]:


def ggw(x):
    return torch.sin(3.0 * x)+ torch.sin(0.3 * x)+  torch.sin(0.03 * x)

def morlet_wavelet(x):
#     print(x)
    return (torch.cos(1.75 * x)) * torch.exp(-x**2/2)

def gaussian_wavelet(x,sigma = 1, omega = 0.5 ):
    return torch.exp(-0.5 * (x ** 2) / (sigma** 2)) * torch.cos(omega * x)

def mexhican_hat_wavelet(x, sigma=2, omega=0.577):
    return (torch.pi** (-0.25)*sigma*omega)*(1-x**2)*(torch.exp(-0.5 * (-x**2))) 

def shannon(x):
    return torch.sin(torch.pi *(x-0.5)) - torch.sin(2*(x-0.5) * torch.pi) *(1/torch.pi*(x-0.5))

In [18]:
#shannon(torch.tensor(3.4))

In [19]:
import math
import torch
import torch.nn as nn
import numpy as np

In [20]:
import random

In [21]:
rx=torch.randn(23,50)
weights = torch.Tensor(rx)
print("weights_Encoder",weights)
print("weights_Encoder_shape",weights.shape)

weights_Encoder tensor([[ 1.3372e+00, -1.4239e+00,  1.8790e+00,  ..., -3.7782e-01,
         -1.8287e-01, -5.6388e-01],
        [-2.0979e-01,  9.1405e-01,  2.3676e-01,  ...,  5.2883e-01,
          1.0056e+00,  3.2725e-01],
        [-1.5212e-01,  1.9923e+00, -1.9311e+00,  ...,  9.0922e-01,
          1.7450e-01, -6.6999e-01],
        ...,
        [ 1.2353e+00, -4.2787e-01,  2.8472e-01,  ...,  6.8493e-01,
         -3.8260e+00,  7.9895e-01],
        [-4.0318e-01,  1.1398e-01, -2.0358e+00,  ...,  6.7130e-01,
          5.2907e-01, -3.7280e-01],
        [ 9.4088e-01,  1.7632e-03,  2.4348e-01,  ...,  2.0190e+00,
          5.5376e-01, -1.5671e-03]])
weights_Encoder_shape torch.Size([23, 50])


In [22]:
weights.t().shape

torch.Size([50, 23])

In [23]:
    x = torch.randn(50)
    torch.Tensor(x).reshape(1,-1)

tensor([[-0.1580,  0.7402,  0.4280, -0.7012, -0.6430,  0.2528,  0.6952,  1.1108,
         -1.3987, -0.6881, -0.2364,  0.3753, -0.1826,  1.4371, -1.8403,  0.3403,
         -2.1903, -1.1082,  1.1232,  0.4737,  1.4747, -0.3415, -1.4462,  1.0004,
         -0.0087, -1.1245, -0.0549,  0.3759, -0.4917,  0.3742,  1.1552,  1.6551,
          0.0372, -1.2024,  0.7052, -1.2852, -0.1183,  0.3929, -2.3329,  2.3912,
          0.3313, -0.0394,  0.1156,  2.2958, -0.0735, -3.0611,  0.5383, -0.8258,
          0.2341,  0.1108]])

In [208]:
x[0]

tensor(-0.1580)

## VAE

In [209]:
class VAE2(nn.Module):
    def __init__(self,D_in,latent_dim,fn,H=10,H2=15):

        #Encoder
        super(VAE2,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)

#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)

#         # Decoder
        self.linear4=nn.Linear(H2,H)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H)
        self.linear5=nn.Linear(H,D_in)
        self.lin_bn5 = nn.BatchNorm1d(num_features=D_in)
        self.prev_seed =random.uniform(0,1)

        self.relu = nn.ReLU()
        self.Tanh = nn.Tanh()
        
        self.fn=fn

    def encode(self, x):
        
        if self.fn=='tanh':
            lin1 = self.Tanh(self.lin_bn1(self.linear1(x)))
            lin2 =self.Tanh(self.lin_bn2(self.linear2(lin1)))
            
        if self.fn=='relu':
            lin1 = self.relu(self.lin_bn1(self.linear1(x)))
            lin2 =self.relu(self.lin_bn2(self.linear2(lin1)))
            

        fc1 = F.relu(self.bn1(self.fc1(lin2)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2

    def decode(self, z):
        
        if self.fn=='tanh':
            fc3 = self.Tanh(self.fc_bn3(self.fc3(z)))
            fc4 =self.Tanh(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.Tanh(self.lin_bn4(self.linear4(fc4)))
            #lin5 = self.Tanh(self.lin_bn5(self.linear5(lin4)))
        if self.fn=='relu':
            fc3 = self.relu(self.fc_bn3(self.fc3(z)))
            fc4 =self.relu(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.relu(self.lin_bn4(self.linear4(fc4)))
            #lin5 = self.Tanh(self.lin_bn5(self.linear5(lin4)))
        
        return self.lin_bn5(self.linear5(lin4))
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu,sigma = self.encode(x)
        prev_seed = self.prev_seed
#         print('vae prev_seed',prev_seed)
        z = self.reparameterize(mu,sigma)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decode(z),mu,sigma

In [210]:
class VAE3(nn.Module):
    def __init__(self,D_in,latent_dim,fn,H=10,H2=15):

        #Encoder
        super(VAE3,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)

#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)

#         # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        self.prev_seed =random.uniform(0,1)

        self.relu = nn.ReLU()
        self.Tanh = nn.Tanh()
        
        self.fn=fn

    def encode(self, x):
        
        if self.fn=='tanh':
            lin1 = self.Tanh(self.lin_bn1(self.linear1(x)))
            lin2 =self.Tanh(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.Tanh(self.lin_bn3(self.linear3(lin2)))
        if self.fn=='relu':
            lin1 = self.relu(self.lin_bn1(self.linear1(x)))
            lin2 =self.relu(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2

    def decode(self, z):
        
        if self.fn=='tanh':
            fc3 = self.Tanh(self.fc_bn3(self.fc3(z)))
            fc4 =self.Tanh(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.Tanh(self.lin_bn4(self.linear4(fc4)))
            lin5 = self.Tanh(self.lin_bn5(self.linear5(lin4)))
        if self.fn=='relu':
            fc3 = self.relu(self.fc_bn3(self.fc3(z)))
            fc4 =self.relu(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.relu(self.lin_bn4(self.linear4(fc4)))
            lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        
        return self.lin_bn6(self.linear6(lin5))
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu,sigma = self.encode(x)
        prev_seed = self.prev_seed
#         print('vae prev_seed',prev_seed)
        z = self.reparameterize(mu,sigma)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decode(z),mu,sigma

In [211]:
class VAE4(nn.Module):
    def __init__(self,D_in,latent_dim,fn,H=10,H2=15,H3=20):

        #Encoder
        super(VAE4,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H3)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H3)
        self.linear4=nn.Linear(H3,H3)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H3)
        

#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H3, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H3)
        self.fc_bn4 = nn.BatchNorm1d(H3)

#         # Decoder
        self.linear5=nn.Linear(H3,H3)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H3)
        self.linear6=nn.Linear(H3,H2)
        self.lin_bn6 = nn.BatchNorm1d(num_features=H2)
        self.linear7=nn.Linear(H2,H)
        self.lin_bn7 = nn.BatchNorm1d(num_features=H)
        self.linear8=nn.Linear(H,D_in)
        self.lin_bn8 = nn.BatchNorm1d(num_features=D_in)
        self.prev_seed =random.uniform(0,1)

        self.relu = nn.ReLU()
        self.Tanh = nn.Tanh()
        
        self.fn=fn

    def encode(self, x):
        
        if self.fn=='tanh':
            lin1 = self.Tanh(self.lin_bn1(self.linear1(x)))
            lin2 =self.Tanh(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.Tanh(self.lin_bn3(self.linear3(lin2)))
            lin4 = self.Tanh(self.lin_bn4(self.linear4(lin3)))
        if self.fn=='relu':
            lin1 = self.relu(self.lin_bn1(self.linear1(x)))
            lin2 =self.relu(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))
            lin4 = self.relu(self.lin_bn4(self.linear4(lin3)))

        fc1 = F.relu(self.bn1(self.fc1(lin4)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2

    def decode(self, z):
        
        if self.fn=='tanh':
            fc3 = self.Tanh(self.fc_bn3(self.fc3(z)))
            fc4 =self.Tanh(self.fc_bn4(self.fc4(fc3)))

            lin5 =self.Tanh(self.lin_bn5(self.linear5(fc4)))
            lin6 = self.Tanh(self.lin_bn6(self.linear6(lin5)))
            lin7 = self.Tanh(self.lin_bn7(self.linear7(lin6)))
        if self.fn=='relu':
            fc3 = self.relu(self.fc_bn3(self.fc3(z)))
            fc4 =self.relu(self.fc_bn4(self.fc4(fc3)))

            lin5 =self.relu(self.lin_bn5(self.linear5(fc4)))
            lin6 = self.relu(self.lin_bn6(self.linear6(lin5)))
            lin7 = self.relu(self.lin_bn7(self.linear7(lin6)))
        
        return self.lin_bn8(self.linear8(lin7))
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu,sigma = self.encode(x)
        prev_seed = self.prev_seed
#         print('vae prev_seed',prev_seed)
        z = self.reparameterize(mu,sigma)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decode(z),mu,sigma

In [212]:
class sampling_vector(nn.Module):
    def __init__(self, z,H2):
        super().__init__()
        self.z = z
        self.fc3 = nn.Linear(self.z,H2)
        self.fc_bn3 = nn.BatchNorm1d(H2)
#         self.fc4 = nn.Linear(latent_dims, H2)
#         self.fc_bn4 = nn.BatchNorm1d(H2)
        self.relu = nn.ReLU()
    def forward(self,z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        return fc3

In [213]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")

    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [214]:
epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []


def train(epoch):
    model.train()
    train_loss = 0
    
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if epoch % 100 == 0:
            print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))
        
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 100 == 0:
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [215]:
loss_mse = customLoss()

In [216]:
D_in = trainloader.dataset.x.shape[1]

model = VAE2(D_in,2,'relu').to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [217]:
start = time.time()
epochs=300
for epoch in range(1, epochs + 1):
    train(epoch)
#     test(epoch)
elapsed_time = time.time()-start
print("total_time taken is :",elapsed_time)
#     test(epoch)

====> Epoch: 100 Average training loss: 6.5007
====> Epoch: 200 Average training loss: 5.3169
====> Epoch: 300 Average training loss: 4.5437
total_time taken is : 25.808106184005737


In [218]:
print(train_losses[len(train_losses)-1])

4.543669921875


## HYPER-PARAMETER TUNING

In [219]:
global model
global optimizer

model = VAE2(D_in,2,'relu').to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [220]:
epochs=[1800]
hl=[2,3,4]
lr=[0.0001,0.01,0.001]
mm=[0.001,0.01]
act=['relu','tanh']
opt=['adam','adagrad','sgd']
ldm=[2,3,4]

min_loss=10000
MODEL=None
BESTCOMB=None

for e in epochs:
    for h in hl:
        for l in lr:
            for m in mm:
                for a in act:
                    for o in opt:
                        for ld in ldm:
                            
                            D_in = trainloader.dataset.x.shape[1]
                            
                            if hl==2:
                                model = VAE2(D_in,ld,a).to(device)
                            if hl==3:
                                model = VAE3(D_in,ld,a).to(device)
                            if hl==4:
                                model = VAE4(D_in,ld,a).to(device)
                            
                            if opt=='adam':
                                optimizer = optim.Adam(model.parameters(), lr=l)
                            if opt=='sgd':
                                optimizer = optim.SGD(model.parameters(), lr=l,momentum=m)
                            if opt=='adagrad':
                                optimizer = torch.optim.Adagrad(model.parameters(), lr = l)
                            
                            start = time.time()
                            for epoch in range(1, e+1):
                                train(epoch)
                            #     test(epoch)
                            elapsed_time = time.time()-start
                            #print("total_time taken is :",elapsed_time)

                            cur_loss=train_losses[len(train_losses)-1]
                            if cur_loss < min_loss:
                                min_loss=cur_loss
                                MODEL=model
                                BESTCOMB=[e,h,l,m,a,o,ld]
                            
                            
                            ## junk --ignore
                            model = VAE2(D_in,2,'relu').to(device)
                            optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

====> Epoch: 100 Average training loss: 6.5124
====> Epoch: 200 Average training loss: 5.3319
====> Epoch: 300 Average training loss: 4.5747
====> Epoch: 400 Average training loss: 4.0211
====> Epoch: 500 Average training loss: 3.6102
====> Epoch: 600 Average training loss: 3.2601
====> Epoch: 700 Average training loss: 2.9828
====> Epoch: 800 Average training loss: 2.7482
====> Epoch: 900 Average training loss: 2.5478
====> Epoch: 1000 Average training loss: 2.3659
====> Epoch: 1100 Average training loss: 2.2119
====> Epoch: 1200 Average training loss: 2.0804
====> Epoch: 1300 Average training loss: 1.9603
====> Epoch: 1400 Average training loss: 1.8624
====> Epoch: 1500 Average training loss: 1.7757
====> Epoch: 1600 Average training loss: 1.6861
====> Epoch: 1700 Average training loss: 1.5988
====> Epoch: 1800 Average training loss: 1.5313
====> Epoch: 100 Average training loss: 6.5057
====> Epoch: 200 Average training loss: 5.3293
====> Epoch: 300 Average training loss: 4.5603
====

====> Epoch: 1200 Average training loss: 2.0524
====> Epoch: 1300 Average training loss: 1.9504
====> Epoch: 1400 Average training loss: 1.8642
====> Epoch: 1500 Average training loss: 1.7687
====> Epoch: 1600 Average training loss: 1.6662
====> Epoch: 1700 Average training loss: 1.5971
====> Epoch: 1800 Average training loss: 1.5352
====> Epoch: 100 Average training loss: 6.7445
====> Epoch: 200 Average training loss: 5.4593
====> Epoch: 300 Average training loss: 4.6256
====> Epoch: 400 Average training loss: 4.0711
====> Epoch: 500 Average training loss: 3.6547
====> Epoch: 600 Average training loss: 3.2907
====> Epoch: 700 Average training loss: 3.0161
====> Epoch: 800 Average training loss: 2.7609
====> Epoch: 900 Average training loss: 2.5589
====> Epoch: 1000 Average training loss: 2.3853
====> Epoch: 1100 Average training loss: 2.2376
====> Epoch: 1200 Average training loss: 2.0980
====> Epoch: 1300 Average training loss: 1.9823
====> Epoch: 1400 Average training loss: 1.8718
=

====> Epoch: 500 Average training loss: 3.6114
====> Epoch: 600 Average training loss: 3.2698
====> Epoch: 700 Average training loss: 2.9991
====> Epoch: 800 Average training loss: 2.7637
====> Epoch: 900 Average training loss: 2.5526
====> Epoch: 1000 Average training loss: 2.3764
====> Epoch: 1100 Average training loss: 2.2256
====> Epoch: 1200 Average training loss: 2.0946
====> Epoch: 1300 Average training loss: 1.9808
====> Epoch: 1400 Average training loss: 1.8580
====> Epoch: 1500 Average training loss: 1.7663
====> Epoch: 1600 Average training loss: 1.6796
====> Epoch: 1700 Average training loss: 1.6064
====> Epoch: 1800 Average training loss: 1.5398
====> Epoch: 100 Average training loss: 6.6074
====> Epoch: 200 Average training loss: 5.3640
====> Epoch: 300 Average training loss: 4.5959
====> Epoch: 400 Average training loss: 4.0347
====> Epoch: 500 Average training loss: 3.6204
====> Epoch: 600 Average training loss: 3.2704
====> Epoch: 700 Average training loss: 2.9861
====

====> Epoch: 1600 Average training loss: 1.4673
====> Epoch: 1700 Average training loss: 1.3806
====> Epoch: 1800 Average training loss: 1.2933
====> Epoch: 100 Average training loss: 7.0037
====> Epoch: 200 Average training loss: 5.6370
====> Epoch: 300 Average training loss: 4.8149
====> Epoch: 400 Average training loss: 4.2061
====> Epoch: 500 Average training loss: 3.7345
====> Epoch: 600 Average training loss: 3.3690
====> Epoch: 700 Average training loss: 3.0595
====> Epoch: 800 Average training loss: 2.8075
====> Epoch: 900 Average training loss: 2.5946
====> Epoch: 1000 Average training loss: 2.4212
====> Epoch: 1100 Average training loss: 2.2608
====> Epoch: 1200 Average training loss: 2.1273
====> Epoch: 1300 Average training loss: 2.0178
====> Epoch: 1400 Average training loss: 1.8921
====> Epoch: 1500 Average training loss: 1.7775
====> Epoch: 1600 Average training loss: 1.6924
====> Epoch: 1700 Average training loss: 1.6221
====> Epoch: 1800 Average training loss: 1.5544
=

====> Epoch: 900 Average training loss: 2.5386
====> Epoch: 1000 Average training loss: 2.3639
====> Epoch: 1100 Average training loss: 2.2077
====> Epoch: 1200 Average training loss: 2.0959
====> Epoch: 1300 Average training loss: 1.9548
====> Epoch: 1400 Average training loss: 1.8512
====> Epoch: 1500 Average training loss: 1.7649
====> Epoch: 1600 Average training loss: 1.6783
====> Epoch: 1700 Average training loss: 1.6140
====> Epoch: 1800 Average training loss: 1.5393
====> Epoch: 100 Average training loss: 6.4708
====> Epoch: 200 Average training loss: 5.3164
====> Epoch: 300 Average training loss: 4.6067
====> Epoch: 400 Average training loss: 4.0276
====> Epoch: 500 Average training loss: 3.5926
====> Epoch: 600 Average training loss: 3.2831
====> Epoch: 700 Average training loss: 2.9803
====> Epoch: 800 Average training loss: 2.6482
====> Epoch: 900 Average training loss: 2.4917
====> Epoch: 1000 Average training loss: 2.3517
====> Epoch: 1100 Average training loss: 2.1908
==

...

====> Epoch: 1400 Average training loss: 1.7647
====> Epoch: 1500 Average training loss: 1.6716
====> Epoch: 1600 Average training loss: 1.5873
====> Epoch: 1700 Average training loss: 1.5206
====> Epoch: 1800 Average training loss: 1.4564
====> Epoch: 100 Average training loss: 6.8521
====> Epoch: 200 Average training loss: 5.5843
====> Epoch: 300 Average training loss: 4.7856
====> Epoch: 400 Average training loss: 4.1579
====> Epoch: 500 Average training loss: 3.7166
====> Epoch: 600 Average training loss: 3.3438
====> Epoch: 700 Average training loss: 3.0556
====> Epoch: 800 Average training loss: 2.7783
====> Epoch: 900 Average training loss: 2.5854
====> Epoch: 1000 Average training loss: 2.4012
====> Epoch: 1100 Average training loss: 2.2460
====> Epoch: 1200 Average training loss: 2.0970
====> Epoch: 1300 Average training loss: 1.9860
====> Epoch: 1400 Average training loss: 1.8797
====> Epoch: 1500 Average training loss: 1.7569
====> Epoch: 1600 Average training loss: 1.6880
=

====> Epoch: 700 Average training loss: 3.0584
====> Epoch: 800 Average training loss: 2.8043
====> Epoch: 900 Average training loss: 2.5963
====> Epoch: 1000 Average training loss: 2.4216
====> Epoch: 1100 Average training loss: 2.2533
====> Epoch: 1200 Average training loss: 2.1241
====> Epoch: 1300 Average training loss: 1.9360
====> Epoch: 1400 Average training loss: 1.8906
====> Epoch: 1500 Average training loss: 1.7990
====> Epoch: 1600 Average training loss: 1.7007
====> Epoch: 1700 Average training loss: 1.6298
====> Epoch: 1800 Average training loss: 1.5469
====> Epoch: 100 Average training loss: 6.5628
====> Epoch: 200 Average training loss: 5.3367
====> Epoch: 300 Average training loss: 4.5845
====> Epoch: 400 Average training loss: 4.0145
====> Epoch: 500 Average training loss: 3.5934
====> Epoch: 600 Average training loss: 3.2580
====> Epoch: 700 Average training loss: 2.9534
====> Epoch: 800 Average training loss: 2.7429
====> Epoch: 900 Average training loss: 2.5527
====

====> Epoch: 1800 Average training loss: 1.5416
====> Epoch: 100 Average training loss: 6.5664
====> Epoch: 200 Average training loss: 5.3412
====> Epoch: 300 Average training loss: 4.5796
====> Epoch: 400 Average training loss: 4.0335
====> Epoch: 500 Average training loss: 3.5945
====> Epoch: 600 Average training loss: 3.2756
====> Epoch: 700 Average training loss: 2.9932
====> Epoch: 800 Average training loss: 2.7547
====> Epoch: 900 Average training loss: 2.5597
====> Epoch: 1000 Average training loss: 2.3859
====> Epoch: 1100 Average training loss: 2.2235
====> Epoch: 1200 Average training loss: 2.0856
====> Epoch: 1300 Average training loss: 1.9702
====> Epoch: 1400 Average training loss: 1.8702
====> Epoch: 1500 Average training loss: 1.7700
====> Epoch: 1600 Average training loss: 1.6903
====> Epoch: 1700 Average training loss: 1.6094
====> Epoch: 1800 Average training loss: 1.5508
====> Epoch: 100 Average training loss: 6.5276
====> Epoch: 200 Average training loss: 5.3485
===

====> Epoch: 1100 Average training loss: 2.2247
====> Epoch: 1200 Average training loss: 2.0842
====> Epoch: 1300 Average training loss: 1.9843
====> Epoch: 1400 Average training loss: 1.8604
====> Epoch: 1500 Average training loss: 1.7785
====> Epoch: 1600 Average training loss: 1.6882
====> Epoch: 1700 Average training loss: 1.6136
====> Epoch: 1800 Average training loss: 1.5442
====> Epoch: 100 Average training loss: 6.6367
====> Epoch: 200 Average training loss: 5.4364
====> Epoch: 300 Average training loss: 4.6306
====> Epoch: 400 Average training loss: 4.0695
====> Epoch: 500 Average training loss: 3.6331
====> Epoch: 600 Average training loss: 3.2742
====> Epoch: 700 Average training loss: 2.9946
====> Epoch: 800 Average training loss: 2.7519
====> Epoch: 900 Average training loss: 2.5427
====> Epoch: 1000 Average training loss: 2.3672
====> Epoch: 1100 Average training loss: 2.2199
====> Epoch: 1200 Average training loss: 2.0848
====> Epoch: 1300 Average training loss: 1.9684
=

====> Epoch: 400 Average training loss: 4.0462
====> Epoch: 500 Average training loss: 3.6043
====> Epoch: 600 Average training loss: 3.2675
====> Epoch: 700 Average training loss: 2.9781
====> Epoch: 800 Average training loss: 2.7436
====> Epoch: 900 Average training loss: 2.5481
====> Epoch: 1000 Average training loss: 2.3724
====> Epoch: 1100 Average training loss: 2.2263
====> Epoch: 1200 Average training loss: 2.0786
====> Epoch: 1300 Average training loss: 1.9691
====> Epoch: 1400 Average training loss: 1.8663
====> Epoch: 1500 Average training loss: 1.7690
====> Epoch: 1600 Average training loss: 1.6896
====> Epoch: 1700 Average training loss: 1.6177
====> Epoch: 1800 Average training loss: 1.5347
====> Epoch: 100 Average training loss: 6.5096
====> Epoch: 200 Average training loss: 5.3429
====> Epoch: 300 Average training loss: 4.5936
====> Epoch: 400 Average training loss: 4.0547
====> Epoch: 500 Average training loss: 3.6164
====> Epoch: 600 Average training loss: 3.2792
====

In [203]:
BESTCOMB

In [221]:
MODEL

VAE2(
  (linear1): Linear(in_features=8, out_features=10, bias=True)
  (lin_bn1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=10, out_features=15, bias=True)
  (lin_bn2): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=15, out_features=2, bias=True)
  (bn1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc21): Linear(in_features=2, out_features=2, bias=True)
  (fc22): Linear(in_features=2, out_features=2, bias=True)
  (fc3): Linear(in_features=2, out_features=2, bias=True)
  (fc_bn3): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=2, out_features=15, bias=True)
  (fc_bn4): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=15, out_features=10, bias=True)
  (lin_bn4): BatchNorm1d(10, eps=1e-05, momentum=0.1

In [222]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)


In [223]:
x_hat

[tensor([[ 0.6823,  0.1817,  0.1601,  ...,  0.9453,  0.1416,  0.5184],
         [ 0.6967,  0.7846,  0.3372,  ...,  0.8046,  0.2936, -0.0058],
         [ 0.3732,  0.1750,  0.8032,  ...,  0.2196,  0.8473,  0.5171],
         ...,
         [ 0.4364,  0.2931,  0.6242,  ...,  0.4735,  0.6443,  0.4036],
         [ 0.4676,  0.3111,  0.5840,  ...,  0.5264,  0.6010,  0.4048],
         [ 0.6528,  0.8518,  0.5190,  ...,  0.6587,  0.4759, -0.0767]])]

In [224]:
d1 = pd.DataFrame(np.array(x_hat[0]))

In [225]:
d1

,0,1,2,3,4,5,6,7
0,0.682302,0.181664,0.160124,0.218293,0.087439,0.945278,0.141607,0.518402
1,0.696721,0.784630,0.337177,0.591466,-0.119760,0.804641,0.293623,-0.005806
2,0.373250,0.174955,0.803203,0.405306,0.360827,0.219641,0.847347,0.517073
3,0.380489,-0.113510,0.660653,0.303539,0.453734,0.389440,0.711760,0.696993
4,0.756398,-0.052024,-0.135941,-0.354321,0.075660,1.169491,-0.164971,1.026090
...,...,...,...,...,...,...,...,...
6995,0.373250,0.174955,0.803203,0.405306,0.360827,0.219641,0.847347,0.517073
6996,0.670697,0.306268,0.235417,0.361252,0.061206,0.887190,0.214945,0.362840
6997,0.436424,0.293129,0.624173,0.445553,0.270823,0.473534,0.644261,0.403554
6998,0.467602,0.311101,0.584015,0.435035,0.233166,0.526447,0.601048,0.404837


In [226]:
# d2

In [227]:
org =pd.DataFrame(np.array(trainloader.dataset.x))

In [228]:
# pd.DataFrame(np.array(testloader.dataset.x)).mean()


In [229]:
org.std()

0    0.192962
1    0.141110
2    0.287587
3    0.248447
4    0.192987
5    0.456147
6    0.499946
7    0.286641
dtype: float32

In [230]:
syn_test = pd.concat([d1],ignore_index = True)

In [231]:
syn_test.shape

(7000, 8)

In [232]:
syn_test.std()

0    0.164533
1    0.321246
2    0.331171
3    0.299163
4    0.194953
5    0.348831
6    0.359857
7    0.323809
dtype: float32

In [233]:
# d1.to_csv('vae-wnn-yes.csv')

## Evasion Attack

In [234]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_test = pd.concat([d1],ignore_index = True)

In [237]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, stratify=y, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

preds = lr.predict(eva_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(X_train,Y_train)

preds =logi.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#        best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(X_train,Y_train)
preds=gb.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train,Y_train)
preds = xgb_model.predict(eva_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2389 0 611 0
0.0
1.0
0.5
Decision Tree
2143 246 550 61
0.09983633387888707
0.8970280452071997
0.49843218954304336
LGBM Classifier
2389 0 611 0
0.0
1.0
0.5
XGB Classifier
2375 14 609 2
0.0032733224222585926
0.9941398074508162
0.4987065649365374


## Poison Attack

In [238]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_train = pd.concat([d1],ignore_index = True)

In [239]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(eva_train,Y_train)

preds = lr.predict(X_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(eva_train,Y_train)

preds =logi.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(eva_train,Y_train)
preds=gb.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(eva_train,Y_train)
preds = xgb_model.predict(X_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
1528 888 374 210
0.3595890410958904
0.6324503311258278
0.4960196861108591
Decision Tree
776 1640 234 350
0.5993150684931506
0.3211920529801324
0.46025356073664153
LGBM Classifier
2263 153 531 53
0.09075342465753425
0.9366721854304636
0.513712805043999
XGB Classifier
1595 821 374 210
0.3595890410958904
0.660182119205298
0.5098855801505942
